# Reference input and robust tracking control with state-variables

## Introducing a reference input with full state feedback

In order to study the transient response of the pole placement designs to input commands, it's necessary to introduce the reference input into the system. An obvious way to do this is to change the control to $u = -\pmb K \pmb x + r$. However, the system will now almost surely have a non zero steady-state error to a step input.
If the desired final values of the state and the control input are $\pmb x_{ss}$ and $u_{ss}$ respectively, then the new control formula should be 

\begin{equation}
u = u_{ss} - \pmb K(\pmb x - \pmb x_{ss})
\label{eq:u-steady-state}
\end{equation}

so that when $\pmb x = \pmb x_{ss}$ (no error), $u = u_{ss}$. To pick the correct final values, we must solve the equations so that the system will have zero steady-state error to any constant input. The system differential equations are defined by $\eqref{eq:stat-form}$ and $\eqref{eq:output}$; in the constant steady state they are reduced to the pair:


\begin{align}
\mathbf{0} &= \pmb F \pmb x_{ss} + \pmb G u_{ss}\\
y_{ss} &= \pmb H \pmb x_{ss} + J u_{ss}
\label{eq:ss-equations}
\end{align}






We wnat to solve for values for which $y_{ss} = r_{ss}$. To do this, we make $\pmb{x_{ss}} = \pmb N_x r_{ss}$ and ${u_{ss}} = N_u r_{ss}$. With these substitutions we can write $\eqref{eq:ss-equations}$ as a matrix equation:

\begin{equation}
\begin{bmatrix}
\pmb F & \pmb G\\
\pmb H & J
\end{bmatrix}=
\begin{bmatrix}
\pmb N_x\\
N_u
\end{bmatrix} =
\begin{bmatrix}
\mathbf{0}\\
1
\end{bmatrix}
\label{eq:ss-vector-form}
\end{equation}

This equation can be solved for $\pmb N_x$ and $N_u$ to get:

\begin{equation}
\begin{bmatrix}
\pmb N_x\\
N_u
\end{bmatrix} =
\begin{bmatrix}
\pmb F & \pmb G\\
\pmb H & J
\end{bmatrix}^{-1}
\begin{bmatrix}
\mathbf{0}\\
1
\end{bmatrix}
\label{eq:ss-vector-sol}
\end{equation}

With these values, we finally have the basis for introducing the reference input so as to get zero steady-state error to a step input:

\begin{equation}
u =  N_u - \pmb K(\pmb x - \pmb N_x r) = -\pmb K \pmb x + (N_u + \pmb K \pmb N_x)r
\label{eq:reference-input1}
\end{equation}

The coefficient of r in parentheses is a constant that can be computed beforehand. We give it the symbol $\bar{N}$ so

\begin{equation}
u = -\pmb K \pmb x + \bar N r
\label{eq:reference-input2}
\end{equation}

With the reference input in place, the close loop system has input $r$ and output $y$. From the state description we know that the system poles are at the eigenvalues of the close loop system matrix $\pmb{F-GK}$. In order to compute the close loop transient response, it's necessary to know where the close loop zeros of the transfer function from r to y are.They are to be found applying $\eqref{eq:zerodet}$ to the close loop description, which it's assumed don't have direct path from input $u$ to output $y$, so that $J=0$. The zeros are values of $s$ such that 

\begin{equation}
det\begin{bmatrix}
s\pmb I - (\pmb F - \pmb G \pmb K) & -\bar N \pmb G\\
\pmb H & 0
\end{bmatrix} = 0
\label{eq:zeros}
\end{equation}

It's possible assume two elementary facts about determinants to simplify $\eqref{eq:zeros}$: in the first place if we divide the last column by $\bar N$, then the point where the determinant is zero remains unchanged. The second is that the determinant also not change if it's multiplied the last column by $\pmb K$ and ad it to the first block column; the result is that the $\pmb{GK}$ term is cancelled out. Thus the matrix equation for zeros reduces to

\begin{equation}
det\begin{bmatrix}
s\pmb I - \pmb F & -\pmb G\\
\pmb H & 0
\end{bmatrix} = 0
\label{eq:zeros-simplified}
\end{equation}
The above equation is the same as $\eqref{eq:zerodet}$ for the zeros of the plant *before* the feedback was applied. The important conclusion is:

<p style="font-weight:bold;padding:35px;background-color:azure;text-align:center"> When full-state feedback is used as in $\eqref{eq:reference-input1}$ or $\eqref{eq:reference-input2}$, the zeros remain unchanged by the feedback</p>

## Hand on lab for cart pendulum with reference input
Having described how to obtain precise result it's shown below the block scheme *without* a gain reference. It's expected to obtain an output error.

|                        |
|:----------------------:|
|![HL1.png](images/HL1.png)|

<p style="text-align:center"><a name="fig19">Fig. 19 </a> Block scheme of reference system with Linear Quadratic Regulator</p>

The design criteria for this system with the cart receiving a 0.2 m step input are as follows:
 - Settling time for x and theta of less than 5 seconds.
 - Rise time for x of less than 1 second
 - Overshoot of $\theta$ less than 20 degree (0.35 radians). Please note in this lab the up position is 0 degree and down position is $\pi$
 - Steady-state error within 2%

For an inverted pendulum it is unrealistic to consider just the single output system. Using state-space/state variables methods it is relatively simple to work with a multi-output system, so in this hand-on lab it will design a controller with both the pendulum angle and the cart position in mind.
 

|                                                    |
|:--------------------------------------------------:|
|![1stepresponse.png](images/1stepresponse.png)      |

<p style="text-align:left"><a name="fig20">Fig. 20 </a> The curve in red represents the pendulum's angle, in radians and the curve in blue represents the cart's position in meters. As you can see, this plot is not satisfactory. The pendulum and cart's overshoot appear fine, but their settling times need improvement and the cart's rise time needs to go down.In addition the cart is not near the desired location but has in fact moved in the other direction. This error will be dealt with in the next section and right now we will focus on the settling and rise times.</p>

A piece of code
```
%% Set cart-pendulum parameters
m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = 1; % pendulum up (s=1)
%% Base system Matrices
F = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

G = [0; 1/M; 0; s*1/(M*L)];

% Set cart-position
H = [1 0 0 0; 
        0 0 1 0];  

J = zeros(size(H,1),size(G,2))
```

The Matlab command window should output the following text as a result:

```
>> eig(F)

ans =

         0
   -2.4674
   -0.1665
    2.4339
```

The next step in the design process is to assume to have full-state feedback (i.e. that we can measure all four states), and find the vector $K$ which determines the feedback control law. This can be done in a number of ways. If it's known the desired closed-loop poles, it's possible to use the *place* or *acker* command. Another option is to use the *lqr* function; this will give you the optimal controller (under certain assumptions. The lqr function allows to choose two parameters, R and Q, which will balance the relative importance of the input and state in the cost function to optimize the trade-off. The simplest case is to assume R=1, and Q=H'*H. It's worthwhile to note that it's possible using both outputs (the pendulum's angle and the cart's position). Essentially, the lqr method allows to control both outputs. In this case, it is pretty easy to do. The controller can be tuned by changing the nonzero elements in the Q matrix to get a desirable response.
To find the structure of Q it's possible use the matlab command:
```
>> H'*H

ans =

     1     0     0     0
     0     0     0     0
     0     0     1     0
     0     0     0     0
```
The element in the 1,1 position will be used to weight the cart's position and the element in the 3,3 position will be used to weight the pendulum's angle. The input weighting R will remain at 1. Now that we know what the Q matrix should look like we can experiment to find the K matrix that will give a good controller. Then it's the time to find the K matrix and plot the response all in one step so that changes can be made in the control and be seen automatically in the response. Enter the following text into your m-file: 
```
%% Linear Quadratic Regulator
%% try different value of Q
Q = [1 0 0 0;
    0 0 0 0;
    0 0 1 0;
    0 0 0 0];
R = 1;
K = lqr(F,G,Q,R);  % design controller u = -K*x
```
```
>> K

K =

   -1.0000   -5.4325  153.2343   63.8985
```

As noted in <a href="#fig20">figure 20</a> the curve in red represents the pendulum's angle, in radians and the curve in blue represents the cart's position in meters. As you can see, this plot is not satisfactory. The pendulum and cart's overshoot appear fine, but their settling times need improvement and the cart's rise time needs to go down. The cart is not near the desired location but has in fact moved in the other direction. This error will be dealt with in the next section and right now we will focus on the settling and rise times. Go back to your m-file and change the x and y variables to see if you can get a better response. Increasing x makes the settling and rise times go down, and lowers the angle the pendulum moves. Using x=5000 and y=100, the following value of K and step response were found: 
```
%% Linear Quadratic Regulator
%% try different value of Q
Q = [5000 0 0 0;
    0 0 0 0;
    0 0 100 0;
    0 0 0 0];
R = 1;
K = lqr(F,G,Q,R);  % design controller u = -K*x
```
```
>> K

K =

  -70.7107  -91.3062  636.5903  280.1483
```

|                                            |
|:------------------------------------------:|
|![2stepresponse.png](images/2stepresponse.png)|

<p style="text-align:center"><a name="fig21">Fig. 21 </a> Different set of Q parameters </p>

If it's increased $Q_x$ and $Q_{\theta}$ even higher, it's possible to improve the response even more. The reason this plot was chosen was because it satisfied the design requirements while keeping x and y as small as possible. In this problem, $Q_x$ and $Q_{\theta}$ have been used to describe the relative weight of the tracking error in the cart's position and pendulum's angle versus the control effort. The higher these 2 parameters are, the more control effort is used, but the smaller the tracking error. The system response has a settling time under 3 seconds.

 In contrast to the other design methods, where it's feedback the output and compare it to the reference input to compute an error, with a full-state feedback controller it's feeding back all the states. It necessary to compute what the steady-state value of the states should be, multiply that by the chosen gain $K$, and use a new value as our reference for computing the input. This can be done by adding a constant gain $\bar{N}$ after the reference. The schematic below shows this relationship: 

|             |
|:-----------:|
|![systemwithreference2.png](images/systemwithreference2.png)|

<p style="text-align:center"><a name="fig22">Fig. 22 </a> Adding a gain in a reference input</p>

Nbar can be found using the user-defined function rscale. A different H had to be used because the rscale function will not work for multiple outputs. However, the Nbar found is correct, as you can see from the output below:

|         |
|:-------:|
|![3stepinput.png](images/3stepinput.png)|

<p style="text-align:center"><a name="fig23">Fig. 23 </a> Step response with LQR and $\bar{N}$ control</p>

Here the matlab code:
```
%% Set augmented system
Ac = [(F - G*K)];
Bc = [G];
Cc = [H];
Dc = [J];

%% Plot system with reference
T=0:0.01:25;
U=0.2*ones(size(T));
[Y,X]=lsim(Ac,Bc,Cc,Dc,U,T);
%plot(T,Y)
%legend('Cart','Pendulum')


%% Set gain reference
Cn = [1 0 0 0];
Nbar = rscale(F,G,Cn,0,K);
Bcn = [Nbar*G];
[Y,X] = lsim(Ac,Bcn,Cc,Dc,U,T);
plot(T,Y)
legend('Cart position','pendulum angle')
```
```
function[Nbar]=rscale(A,B,C,D,K)

s = size(A,1);
Z = [zeros([1,s]) 1];
N = inv([A,B;C,D])*Z';
Nx = N(1:s);
Nu = N(1+s);
Nbar=Nu + K*Nx;
```
Note that rscale function implements the equation $\eqref{eq:ss-vector-sol}$

## Robust Tracking Control: The error space Approach

The proble of tracking $r$ and rejecting the disturbance can be seen as an exercise in design a control law to provide *regulation of the error*, which is to say that the error $e$ tends to zero as time gets large. The control must also be **structurally stable** or **robust**, in the sense that regulation of $e$ to zero in the steady-state occurs even in the presence of "small" perturbations of the original system parameters. Note that, in practice, it's impossible to have a perfect model of the plant, and the values of parameters are virtually always subject to some change, so robustness is always very important.

Suppose we have a system equations

\begin{align}
\dot{\pmb x} &= \pmb{Fx} + \pmb G u + \pmb G_1 w \label{eq:state-system} \\
y &= \pmb H \pmb x \label{eq:output-system}
\end{align}

and a reference signal that is known to satisfy a specific differential equation. Plant disturbances of the same class may also be present. The goal is to design a controller for this system so that the close-loop system will have specified poles, and can also track input command signals, and reject disturbances of the type described without steady-state error. A solution is developed only for second order differential equation. Let's define the reference input to satisfy the relation 

\begin{equation}\label{eq:eq-input}
\ddot r + \alpha_1 \dot r + \alpha_2 r = 0
\end{equation}

and the disturbance to satisfy exactly the same equation:
\begin{equation}\label{eq:eq-disturbance}
\ddot w +\alpha_1 \dot w +\alpha_2 w = 0
\end{equation}


The tracking error is defined as
\begin{equation}\label{eq:error}
e = y - r
\end{equation}

The strategy is eliminate the reference from the equations in favor of the error. Replacing $r$ in $\eqref{eq:eq-input}$ with the error of $\eqref{eq:error}$ then it's possible to note that reference cancels because of Eq. $\eqref{eq:eq-input}$ obtaining the below result


\begin{align}
\ddot e + \alpha_1 \dot e + \alpha_2 e &= \ddot y + \alpha_1 \dot y + \alpha_2 y \label{eq:no-input} \\
                                       &= \pmb H \pmb{\ddot x} + \alpha_1 \pmb H \pmb{\dot x}+ \alpha_2 \pmb H     \pmb{x} \nonumber
\end{align}


Now it's possible replace the plant state vector with the error space state, defined by

\begin{equation}\label{eq:xi}
\xi \equiv \pmb{\ddot x} + \alpha_1 \pmb{\dot x} + \alpha_2 \pmb{x}
\end{equation}

and similarly replace the control with *the control in error space* defined by

\begin{equation}\label{eq:control-error}
\mu \equiv {\ddot u} + \alpha_1 {\dot u} + \alpha_2 {u}
\end{equation}

The error differential equation becomes:

\begin{equation}\label{eq:eq-error2}
\ddot e + \alpha_1 \dot e + \alpha_2 e = \pmb H \xi
\end{equation}

and the state equation for $\xi$ is given by

\begin{equation}\label{eq:xi-state-eq}
\dot \xi = \pmb{\dddot x} + \alpha_1 \pmb{\ddot x} + \alpha_2 \pmb{\dot x}
\end{equation}


Notice that the disturbance, as well the reference, cancels from $\eqref{eq:xi-state-eq}$. In standard state-variable form, the equations are :

\begin{equation}\label{eq:state-z}
\pmb{\dot z} = \pmb A \pmb z + \pmb B \mu
\end{equation}

where $\pmb z = \begin{bmatrix}e & \dot e & \xi^T \end{bmatrix}^{T}$ and :

\begin{equation}\label{eq:matrices}
\pmb A = \begin{bmatrix}
0 & 1 & \mathbf{0}\\
-\alpha_2 & -\alpha_1 & \pmb H\\
\mathbf{0} & \mathbf{0} & \pmb F
\end{bmatrix}, \pmb B = \begin{bmatrix}
0\\
0\\
\pmb G
\end{bmatrix}
\end{equation}

Having back the system into a canonical form it may be asked about the controllability. It turns out that the plant $(\pmb F, \pmb G)$ is controllable and does not have a zero at any of the roots of the reference-signal characteristic equation

\begin{equation}\label{eq:reference-signal-ch-eq}
\alpha_r(s) = s^2 + \alpha_1 s + \alpha_2 
\end{equation}

then the error system $(\pmb A, \pmb B)$ is controllable.
Assuming these conditions hold, there exist a control law of the form

\begin{equation}\label{eq:control-error-law}
\mu = -\begin{bmatrix} K_2 & K_1 & \pmb K_0 \end{bmatrix} \begin{bmatrix}e\\ \dot e\\ \xi \end{bmatrix}=-\pmb{Kz}
\end{equation}

such that the error system has arbitrary dynamics by pole placement.

Combining the equations $\eqref{eq:control-error-law}$,$\eqref{eq:xi}$ and $\eqref{eq:control-error}$ to get the control law in term of $u$ and $\pmb x$:
\begin{equation}\label{eq:control-law-u-x}
(u+\pmb K_0 \pmb x)^{(2)} + \sum_{i=1}^{2}{\alpha_i(u+\pmb K_0 \pmb x)}^{(2-i)}=-\sum_{i=1}^{2}{K_ie^{2-1}}
\end{equation}


It's possible to note that for $\dot r=0$ the control law $\eqref{eq:control-law-u-x}$ reduces to:
\begin{equation}\label{eq:PI-diff-eq}
\dot u + \pmb K_0 \dot{\pmb x} = -K_1e
\end{equation}

Integrating for u revealing the control law and the action of integral control:
\begin{equation}\label{eq:integral-control}
u=-K_1 \int_{}^{t}{e d\tau} - \pmb K_0\pmb x
\end{equation}

A block diagram of the system is shown below; in the forward controller is possible to see the presence of a pure integrator.

|      |
|:----:|
|![integral-control.png](images/integral-control.png)|

<p style="text-align:center"><a name="fig24">Fig. 24 </a> Reference with integral control</p>

From the nature of the pole placement problem, the state $\pmb z$ in Eq.$\eqref{eq:state-z}$ will tend toward zero for all perturbations in the system parameters as long as $\pmb{A-BK}$ remains stable. It's clear that the signals rejected are those satisfy the equations with values $\alpha_i$ implemented in the model. Error values means a steady state error also. 

## Hand on lab. Control integrator for inverted pendulum

For the system:
\begin{align}\label{eq:state-eq-control-integral}
\pmb{\dot x} &= \pmb F \pmb x + \pmb G u + \pmb G_1 w \\
y &= \pmb H \pmb x \nonumber
\end{align}

it's possible feed back the integral control, $e=y-r$, together the state of the plant $\pmb x$, by augmenting the plant state with the extra integral state $x_I$ which respond to the differential equation:
\begin{equation}\label{eq:integral-error}
\dot{x_I} = \pmb H \pmb x - r(=e), x_I = \int_{}^{t}{e \space d\tau}
\end{equation}

The augmented state equations become:
\begin{equation}\label{eq:aug-state-eq}
\begin{bmatrix}
\dot x_I\\
\pmb{\dot x}
\end{bmatrix}=
\begin{bmatrix}
0 & \pmb H\\
\mathbf{0} & \pmb F
\end{bmatrix}
\begin{bmatrix}
x_I\\
\pmb x
\end{bmatrix}+
\begin{bmatrix}
0\\
\pmb G
\end{bmatrix}u -
\begin{bmatrix}
1\\
\mathbf{0}
\end{bmatrix} r +
\begin{bmatrix}
0\\
\pmb{G}_1
\end{bmatrix} w
\end{equation}

and the feedback law is:
\begin{equation}\label{eq:feedback-law}
u = - \begin{bmatrix}
K_I & \pmb K_0
\end{bmatrix}
\begin{bmatrix}
x_I\\
\pmb x
\end{bmatrix}
\end{equation}

The problem is traced back to a LQR system.

|           |
|:---------:|
|![simulink-integral-control.png](images/simulink-integral-control.png)|

<p style="text-align:center"><a name="fig25">Fig. 25 </a> Simulink for integral control with reference</p>

Building a simulation via *simulink* means recover the LQR part plus the feedback with reference, an integral control plus a random number generator to create the disturbance over the *u* line. In particular the system is under a unit step reference that cart pendulum position "x" has to follow.
The matlab code follow the LQR part for the augmented system:
```
clear all, close all, clc

%% Set cart-pendulum parameters
m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = 1; % pendulum up (s=1)
%% Base system Matrices
F = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

G = [0; 1/M; 0; s*1/(M*L)];

% Set cart-position and pendulum angle
H = [1 0 0 0]
        

J = zeros(size(H,1),size(G,2));

%% Integral control augmented system
Fa = [0 H;0 F(1,:);0 F(2,:);0 F(3,:);0 F(4,:)];
Ga = [0; G];
K = lqr(Fa,Ga,eye(5),R);
```
printing K via command line

```
>> K

K =

   1.0e+03 *

   -0.1000   -0.2696   -0.3145    2.0341    0.9096
```

Here the graphs of simulation:

|         |
|:-------:|
|![integral-control-with-reference.png](images/integral-control-with-reference.png)|

<p style="text-align:left"><a name="fig26">Fig. 26 </a>As noted $\theta=\pi$ when pendulum is in "up" unstable position. The feed back reject the disturbance and the cart tracks the step reference starting at time $t=10$ seconds </p>